# Лабораторная работа 4.
# Методы поиска.


## Выполнил студент группы БВТ2101 Горский Максим Сергеевич
***

### Задание
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

1.  Бинарный поиск, бинарное дерево, Фибоначчиев, интерполяционный

2.  Простое хэширование, рехэширование с помощью псевдослучайных чисел, метод цепочек

3.  Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого». Подразумевается, что ферзь бьёт все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям. Написать программу, которая находит хотя бы один способ решения задач.




### Выполнение:

#### Задание 1:

###### Бинарный поиск:


In [21]:
def binary_find(mas, number):
    left, right = 0, len(mas)-1  # Указатели границ, внутри которых мы смотрим средний элемент
    while right >= left:
        middle = left + (right-left) // 2  # Находим середину в промежутке между left и right
        if mas[middle] == number:
            return middle
        elif mas[middle] < number:
            if left != middle:  # Добавочное условие для нахождения крайних элементов
                left = middle
            else:
                left += 1
        elif mas[middle] > number:
            if right != middle:
                right = middle
            else:
                right -= 1
    return None

###### Бинарное дерево:


In [22]:
class BinarySearchTree:
    """
    Отвечает за организацию дерева в целом: хранит корень, осуществляет вставку новых узлов
    """

    def __init__(self):
        self.root = None
        self.size = 0

    def length(self):
        return self.size

    def __len__(self):
        return self.size

    def __setitem__(self, k, v):
        self.put(k, v)

    def put(self, key, val=None):
        if self.root:
            self._put(key, self.root, val)
        else:
            self.root = TreeNode(key, val)
        self.size = self.size + 1

    def _put(self, key, current_node, val=None):
        if key < current_node.key:
            if current_node.has_left_child():
                self._put(key, current_node.leftChild, val)
            else:
                current_node.leftChild = TreeNode(key, val, parent=current_node)
        else:
            if current_node.has_right_child():
                self._put(key, current_node.rightChild, val)
            else:
                current_node.rightChild = TreeNode(key, val, parent=current_node)

    def get(self, key):
        if self.root:
            res = self._get(key, self.root)
            if res:
                return res.payload
            else:
                return None
        else:
            return None

    def _get(self, key, currentNode):
        if not currentNode:
            return None
        elif currentNode.key == key:
            return currentNode
        elif key < currentNode.key:
            return self._get(key, currentNode.leftChild)
        else:
            return self._get(key, currentNode.rightChild)

    def __getitem__(self, key):
        return self.get(key)

    def __contains__(self, key):
        if self._get(key, self.root):
            return True
        else:
            return False

    def delete(self, key):
        if self.size > 1:
            node_to_remove = self._get(key, self.root)
            if node_to_remove:
                self.remove(node_to_remove)
                self.size = self.size - 1
            else:
                raise KeyError('Error, key not in tree')
        elif self.size == 1 and self.root.key == key:
            self.root = None
            self.size = self.size - 1
        else:
            raise KeyError('Error, key not in tree')

    def __delitem__(self, key):
        self.delete(key)

    def remove(self, currentNode):
        if currentNode.is_leaf():  # Если нет потомков
            if currentNode == currentNode.parent.leftChild:
                currentNode.parent.leftChild = None
            else:
                currentNode.parent.rightChild = None
        elif currentNode.has_both_children():  # Если оба потомка
            succ = currentNode.find_successor()
            succ.splice_out()
            currentNode.key = succ.key
            currentNode.payload = succ.payload

        else:  # Если один потомок
            if currentNode.has_left_child():
                if currentNode.is_left_child():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.leftChild
                elif currentNode.is_right_child():
                    currentNode.leftChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.leftChild
                else:
                    currentNode.replace_node_data(currentNode.leftChild.key,
                                                  currentNode.leftChild.payload,
                                                  currentNode.leftChild.leftChild,
                                                  currentNode.leftChild.rightChild)
            else:
                if currentNode.is_left_child():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.leftChild = currentNode.rightChild
                elif currentNode.is_right_child():
                    currentNode.rightChild.parent = currentNode.parent
                    currentNode.parent.rightChild = currentNode.rightChild
                else:
                    currentNode.replace_node_data(currentNode.rightChild.key,
                                                  currentNode.rightChild.payload,
                                                  currentNode.rightChild.leftChild,
                                                  currentNode.rightChild.rightChild)


def __delitem__(self, key):
    self.delete(key)


class TreeNode:
    """
    Отвечает за конкретный узел: может быть инициализирован, хранит ссылки на предков и потомков
    """

    def __init__(self, key, val=None, left=None, right=None, parent=None):
        self.key = key
        self.payload = val
        self.leftChild = left
        self.rightChild = right
        self.parent = parent

    def has_left_child(self):
        return self.leftChild

    def has_right_child(self):
        return self.rightChild

    def is_left_child(self):
        return self.parent and self.parent.leftChild == self

    def is_right_child(self):
        return self.parent and self.parent.rightChild == self

    def is_root(self):
        return not self.parent

    def is_leaf(self):
        return not (self.rightChild or self.leftChild)

    def has_any_children(self):
        return self.rightChild or self.leftChild

    def has_both_children(self):
        return self.rightChild and self.leftChild

    def replace_node_data(self, key, value, lc, rc):
        self.key = key
        self.payload = value
        self.leftChild = lc
        self.rightChild = rc
        if self.has_left_child():
            self.leftChild.parent = self
        if self.has_right_child():
            self.rightChild.parent = self

    def find_successor(self):  # Возвращает потомка данного узла
        succ = None
        if self.has_right_child():
            succ = self.rightChild.find_min()
        else:
            if self.parent:
                if self.is_left_child():
                    succ = self.parent
                else:
                    # Если узел - правый потомок своего родителя и сам правого потомка не имеет,
                    # то его преемником будет преемник родителя (исключая сам этот узел).
                    self.parent.rightChild = None
                    succ = self.parent.find_successor()
                    self.parent.rightChild = self
        return succ

    def find_min(self):  # Находит минимальный элемент после себя (самый левый элемент в дереве)
        current = self
        while current.has_left_child():
            current = current.leftChild
        return current

    def splice_out(self):  # Удаление узла (более точечное, чем delete)
        if self.is_leaf():
            if self.is_left_child():
                self.parent.leftChild = None
            else:
                self.parent.rightChild = None
        elif self.has_any_children():
            if self.has_left_child():
                if self.is_left_child():
                    self.parent.leftChild = self.leftChild
                else:
                    self.parent.rightChild = self.leftChild
                self.leftChild.parent = self.parent
            else:
                if self.is_left_child():
                    self.parent.leftChild = self.rightChild
                else:
                    self.parent.rightChild = self.rightChild
                self.rightChild.parent = self.parent

###### Фибоначчиев поиск:

In [23]:
def fibonacci(n):
    fib_seq = [1, 1]
    while len(fib_seq) < n:
        fib_seq.append(fib_seq[-1] + fib_seq[-2])
    return fib_seq


def fibonacci_find(mas, number, fib_seq):
    fib_m = fib_seq[-1]  # Крайний правый член Фибоначчи
    fib_m_1 = fib_seq[-2]  # Первый влево
    fib_m_2 = fib_seq[-3]  # Второй влево
    index = -1  # Левый край блока, где мы ищем number
    while fib_m > 1:
        i = min(index + fib_m_2, (len(mas) - 1))  # Находим предполагаемый индекс
        if mas[i] < number:  # Если элемент по нему меньше чем number
            fib_m = fib_m_1  # То смещаем крайний правый член Фибоначчи на 1 влево
            fib_m_1 = fib_m_2
            fib_m_2 = fib_m - fib_m_1
            index = i  # Перемещаем индекс (По факту двигаем его вправо)
        elif mas[i] > number:  # Если элемент больше чем number
            fib_m = fib_m_2  # То сдвигаем числа Фибоначчи вправо на 2 (По факту двигаем правую границу)
            fib_m_1 = fib_m_1 - fib_m_2
            fib_m_2 = fib_m - fib_m_1
        else:
            return i
    if fib_m_1 and index < (len(mas) - 1) and mas[index + 1] == number:
        return index + 1
    return None

###### Интерполяционный поиск:

In [24]:
def interpolating_find(mas, number):
    left, right = 0, len(mas) - 1  # Указатели границ, внутри которых мы смотрим средний элемент
    while mas[left] < number < mas[right]:
        middle = left + (number - mas[left])*(right-left)//(mas[right]-mas[left])  # Находим новую точку сравнения
        if mas[middle] == number:
            return middle
        elif mas[middle] < number:
            left = middle + 1
        elif mas[middle] > number:
            right = middle - 1
    if mas[left] == number:
        return left
    elif mas[right] == number:
        return right
    return None

#### Задание 2:

###### Простое хэширование:

In [25]:
class MapSimpleHash:

    def __init__(self, length):
        self.length = length
        self.map = [Slot() for _ in range(length)]

    def hash_function(self, val):
        return val % self.length

    def put(self, key, val):
        index = self.hash_function(val)
        self.map[index] = Slot(key, val)

    def get(self, val):
        index = self.hash_function(val)
        return self.map[index].get_key()


class Slot:

    def __init__(self, key=None, val=None):
        self.val = val  # Значение числа
        self.key = key  # Номер в изначальной последовательности

    def get_key(self):
        return self.key

    def get_val(self):
        return self.val

###### Рехэширование с псевдослучайными числами:

In [26]:
class MapReHash:

    def __init__(self, length):
        for i in range(1, 10):
            if length <= 2**i:
                self.length = 2**i
                break
        self.map = [Slot() for _ in range(self.length)]

    def __len__(self):
        return self.length

    def generator_random(self, operations):  # Генератор псевдослучайной последовательности
        r = 1
        pi = 0
        for i in range(operations):
            r = 5 * r
            r = r % (4 * self.length)
            pi = int(r / 4)
        return pi

    def hash_function(self, val):
        return val % self.length

    def rehash_function(self, val, depth):  # Функция рехэширования
        return (self.hash_function(val) + self.generator_random(depth)) % self.length

    def put(self, key, val):
        index = self.hash_function(val)
        if self.map[index].get_key() is None:
            self.map[index] = Slot(key, val)
        else:
            i = 1
            while self.map[index].get_key() is not None:
                index = self.rehash_function(val, i)
                i += 1
            self.map[index] = Slot(key, val)

    def get(self, val):
        index = self.hash_function(val)
        if self.map[index].get_val() == val:
            return self.map[index].get_key()
        else:
            i = 1
            while self.map[index].get_val() != val:
                index = self.rehash_function(val, i)
                i += 1
                if i == self.length:
                    return None
            return self.map[index].get_key()

###### Метод цепочек:

In [27]:
class MapChain:

    def __init__(self, length):
        self.length = length
        self.map = [SlotChain() for _ in range(length)]

    def hash_function(self, val):
        return val % self.length

    def put(self, key, val):
        index = self.hash_function(val)
        if self.map[index].get_key() is None:
            self.map[index] = SlotChain(key, val)
        else:
            self.map[index].put_next_number(key, val)

    def get(self, val):
        index = self.hash_function(val)
        if self.map[index].get_val() == val:
            return self.map[index].get_key()
        else:
            return self.map[index].get_next_number(val)


class SlotChain:

    def __init__(self, key=None, val=None, next_number=None):
        self.val = val  # Значение числа
        self.key = key  # Номер в изначальной последовательности
        self.next_number = next_number

    def put_next_number(self, key, val):
        if self.next_number is None:
            self.next_number = SlotChain(key, val)
        else:
            self.next_number.put_next_number(key, val)

    def get_next_number(self, val):
        if val == self.val:
            return self.key
        elif self.next_number is None:
            return None
        else:
            return self.next_number.get_next_number(val)

    def get_key(self):
        return self.key

    def get_val(self):
        return self.val

    def get_next_number_object(self):
        return self.next_number
